# UCL Python Data Working Group February 2021
## Basic Jupyter Notebook Use Walkthrough
## 1. Introduction to Jupyter

### UCL Computer Science Postgraduate Admissions Example Walkthrough

<img align="right" width="200" height="200" src="https://sammcilroy.github.io/images/cop.jpg?raw=true#right">

Jupyter is an development environment for working interactively with Python (and other languages) and is used heavily in data analysis work. It allows you to combine working Python code with text, images and visualisations making it ideal for exploratory data analysis and report writing/sharing. The combination of code and readable text also make Jupyter a useful tool for documentation of processes. We will use synthesized UCL admissions data to walk through the basic functionality of notebooks including data imports, cleaning and manipulaton, basic analysis and visualisation.

Code is run in individual 'cells' like this one. You can insert and rearrange cells in any order, run them individually or all at once. 

Double click on this cell to inspect it. You can see markdown/HTML text being used to create the titles, text and images in this cell. For more info on markdown a useful resource is https://www.markdowntutorial.com
    



## 2. Setup, Python Libraries

The first code cell in most Jupyter files will be expected to be your imported libraries or any other general setup for your work.

In [ ]:
!pip install -r requirements.txt

from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

## 3 Working with Data, The Pandas Library

The Pandas python library is one of the most common tools for working with dataset in python. It allows for importing of data from various text formats, from databases/SQL queries and from APIs. Data can be manipulated, cleaned and analysed replicating the function of SQL or any typical spreadsheet software which you may be using in your own non-python processes.

## 3.1 Importing Data

We'll use the Pandas library to import data from CSVs containing synthesized UCL Portico (Student Records) Data on Admissions and Offers Made. The data is limited to random courses from Computer Science and Finance to simplify the data.

In [ ]:
'''
Stadard Python comments are also useful when 
a markdown cell would be overkill/cumbersome
'''
# Importing the data...

admissions = pd.read_csv('ucl_cs_pg_admissions_synthesized.csv')
course_codes = pd.read_csv('ucl_course_codes.csv')
cod_codes = pd.read_csv('ucl_country_of_domicile_codes.csv')

## 3.2 Inspecting and Manipulating Data
We can continue using Pandas to inspect our data and make any changes/cleaning necessary before moving on with any analysis...

In [ ]:
# look at data, pandas head() shows the first row(s) of a DataFrame
admissions.head()

In [ ]:
# head() takes integer argument n, default 5, for number of rows to show.
# Pass in argument to change number of rows
course_codes.head(20)

In [ ]:
# tail() argument shows the last (n) rows of a DataFrame
cod_codes.tail()

Inspecting the admissions data....

In [ ]:
# working with data types

'''
When importing data into a DataFrame, unless specified Pandas will assume the data type of each column.
'''

# check the data types of a DataFrame
admissions.dtypes


In [ ]:
'''
We can see that Pandas has interpreted the year and domicile code as integers. The remaining columns
have been assigned to 'object' types meaning they have mixed data types. 
For our analysis we'll want to make sure any codes that could be interpreted as ints are seen as strings
and we can make these changes for individual columns
'''
admissions['STU_CODC'] = admissions['STU_CODC'].astype(str)
admissions.dtypes

In [ ]:
'''
The datatypes are now showing as objects (mixed data types). This is expected as strings in
Python are variable length arrays of characters. We can force the data to be strings of fixed
length but that is not what we want here.

We can also specify/force the data types when importing the data which can be a better practice,
although can become unweildy with larger datasets...
'''
admissions = pd.read_csv('ucl_cs_pg_admissions_synthesized.csv', dtype={'CAP_AYRC': 'int64',
                                                                       'CAP_CRSC': 'str',
                                                                       'STU_GEND': 'str',
                                                                       'STU_CODC': 'str',
                                                                       'CAP_IDRC': 'str'})

admissions.head()

In [ ]:
admissions.dtypes

In [ ]:
# looking at the shape of a DataSet, rowsxcolumns
admissions.shape

Seeing the shape of the imported data we can note we see we have 12751 applications made with 5 dimensions of data on these:

* CAP_AYRC: The year of the application made
* CAP_CRSC: The course (code) applied to
* STU_GEND: The gender of the applicant
* STU_CODC: The country of domicile (code) of the applicant
* CAP_IDRC: The offer made to the applicant and their response

In [ ]:
# look at individual columns with indexing, check unique values
admissions['CAP_AYRC'].unique()

In [ ]:
admissions['CAP_CRSC'].unique()

In [ ]:
len(admissions['CAP_CRSC'].unique())

We note that the data shows applications over 3 years for 12 courses. We'll continue using Pandas to manipulate/clean this admissions data so it is more useful for analysis and sharing...

In [ ]:
# renaming columns, python dictionary

'''
renaming of columns with python dictionary. In Python, names are typically either set to 'snake_case'
or 'Title Case'. Usually snake_case would be best practice for datasets, particualry large ones, meant
for further analysis/storage but we'll use Title Case for here to keep the data 'pretty' and
meaningful for export/reports. Either is acceptable depending on your use case and preferences.
'''

admissions.rename(columns={'CAP_AYRC': 'Year',
                          'CAP_CRSC': 'Course Code',
                          'STU_GEND': 'Gender',
                          'STU_CODC': 'Domicile Code',
                          'CAP_IDRC': 'Decision Response'}, inplace=True) 
                                                                         


    
'''
inplace argument: change the data in place, i.e. modify it. Without this argument, default False,
the change will be done on a temporary copy
'''
admissions.head()

In [ ]:
# lookups/joins 
'''
Python merge function similar to SQL JOIN/Excel LOOKUP.
Use the code lookup tables to replace the course and domicile codes with their actual values
'''
course_codes.rename(columns={"CRS_CODE": 'Course Code'}, inplace=True)
admissions = pd.merge(admissions, course_codes, on='Course Code', how='left')

In [ ]:
cod_codes.rename(columns={'COD_CODE': 'Domicile Code'}, inplace=True)
admissions = pd.merge(admissions, cod_codes, on='Domicile Code', how='left')

admissions.head()

In [ ]:
# reordering columns, typically accomplished by overwriting the data with a new copy with specified columns
admissions = admissions[['Year',
                        'Course Code',
                         'Gender',
                        'CRS_NAME',
                        'Domicile Code',
                        'COD_NAME',
                        'Decision Response']]

admissions.rename(columns={'CRS_NAME': 'Course Name',
                          'COD_NAME': 'Domicile'}, inplace=True)

admissions.head()

In [ ]:
# removing unwanted columns
admissions.drop('Course Code', 1, inplace=True)
admissions.drop('Domicile Code', 1, inplace=True)
admissions.head()

In [ ]:
# mapping one value to another
gender_map = {'M': 'Male', 'F': 'Female'}
admissions['Gender'] = admissions['Gender'].map(gender_map)
admissions.head()

In [ ]:
# sorting the data
admissions.sort_values('Year', ascending=False, inplace=True)

In [ ]:
admissions.head()

In [ ]:
# applying custom functions

'''
define our own basic custom function to add meaning to the decision response codes
'''

def offer_made(decision):
    decision = str(decision)
    if decision.startswith('C'):
        return 'Conditional Offer'
    elif decision.startswith('U'):
        return 'Unconditional Offer'
    elif decision.startswith('R'):
        return 'Rejection'
    else:
        return 'None'
    
admissions['Decision Response'] = admissions['Decision Response'].apply(offer_made)
admissions.rename(columns={'Decision Response': 'Decision'}, inplace=True)
admissions.head()

In [ ]:
# saving the cleaned data to a new csv
admissions.to_csv('admissions_cleaned.csv', index=False)

## 4. Analysing the Data

Once the data is in a cleaned and ready state, we can continue by appying further Pandas/python functions to explore and analyse

In [ ]:
'''
quick analysis by functions
'''
def offer(decision):
    decision = str(decision)
    if 'Conditional' in decision or 'Unconditional' in decision:
        return 1
    return 0

admissions['Offer'] = admissions['Decision'].apply(offer)
admissions.head()

In [ ]:
offers_made = int(admissions['Offer'].sum())
offers_made

In [ ]:
offers_made/len(admissions)

Applying aggregate/sum function to quickly count offers shows that 2818 offers were given which is an overall acceptace rate of 22.1%

In [ ]:
# Segregating the data. Let's look at Msc Computer Science only
# The indexing below shows the data where the rows matching this statement are True
cs = admissions[admissions['Course Name'] == 'MSc Computer Science']
cs.head()

In [ ]:
# we can visualise this by looking at the series our statment creates
array = admissions['Course Name'] == 'MSc Computer Science'
array[0:20]

In [ ]:
len(cs)

In [ ]:
offers_made = int(cs['Offer'].sum())

In [ ]:
offers_made

In [ ]:
acceptance_rate = offers_made/len(cs)
acceptance_rate

MSc Computer Science has 534 offers and a similar acceptance rate of 22.3% over its 2389 applications the past 3 years

In [ ]:
# using HTML in cells

'''
show our findings clearly with HTML formatting...
'''

display(HTML("<font-size:40px><b>MSc Computer Science</b><div style = 'background-color: black; padding: 30px '>" +
              "<span style='color: white; font-size:30px;'> Applications: "  + str(len(cs)) +"</span><p>" +
             "</p><span style='color: white; font-size:30px;'> Offers: "  + str(offers_made) +"</span><p>" +
             "</p><span style='color: green; font-size:30px;'> Acceptance: " + str(round(acceptance_rate*100,1))+'%' + "</span>"+
             "</div>")
       )

In [ ]:
'''
Aggreating/Grouping/Pivoting

Pivot and pivot table functionality in Jupyter to quickly report on data and
see useful patterns. Can be useful in replicating and automating a lot of the
work we see being done in Excel pivot tables...

Let's break down the 534 offers made for MSc Computer Science by Year
'''
cs_offers = cs.groupby(['Year']).sum()
cs_offers

In [ ]:
'''
Create a pivot table of all admissions data, by year, course and count of offers made by Gender
'''
offers_made_by_year_by_gender = pd.pivot_table(admissions, values='Offer', index=['Year', 'Course Name'], columns=['Gender'], aggfunc=np.sum)

In [ ]:
offers_made_by_year_by_gender

In [ ]:
'''
We can then continue working with the data with various options, such as sorting, until we have a table
we are happy with. Let's sort the table to find the courses recruting the most female applicants each year
'''
female_offers = offers_made_by_year_by_gender.reset_index().sort_values(['Year', 'Female'], ascending=[0,0]).set_index(['Year', 'Course Name'])

In [ ]:
female_offers

The sorted tablenow shows the courses by year sorted by most Female offers made...

## 4. Simple Visualisation

Looking at Python plots and visulatisations, let's drill down into the MSc Computer Science Offers...

### 4.1 Basic Python Plots

In [ ]:
'''
Basic Chart/Plot

Show the number of total offers by year. Simple Bar Chart. There are many chart 'kinds' to experiment with
in your own reports.
'''
cs_offers

In [ ]:
cs_offers.plot(kind='bar') # Show Msc Computer Science Offers by Year

In [ ]:
cs_offers.plot(kind='pie', subplots=True) # Show Msc Computer Science Offers by Year

### 4.2 Interactive Charts: Plotly Library

In [ ]:
# make a new dataframe sorting by largest sum of Offers, restrict to top 10, sort descending
top10_domicile = pd.DataFrame(cs.groupby('Domicile')['Offer'].sum().nlargest(10).sort_values(ascending = False))
top10_domicile

In [ ]:
'''
Plotly Scatterplot, with hover and colour coding by Domicile
'''

fig1 = px.scatter(top10_domicile, x = top10_domicile.index, y = 'Offer', size = 'Offer', size_max = 120,
                color = top10_domicile.index, title = 'MSc Computer Science: Top 10 Domiciles by Offers Made')
fig1.show()

Looking at all of the admissions data for 2020...

In [ ]:
'''
Count the applications made in 2020 by course, and sort 
'''
top10_popular_courses_2020 = pd.DataFrame(admissions[admissions['Year'] == 2020].groupby(['Course Name'])['Course Name'].count().sort_values(ascending = True))
top10_popular_courses_2020.rename(columns={'Course Name': 'Applications'}, inplace=True)
top10_popular_courses_2020

In [ ]:
'''
Plotly Bar Chart with Hover and Heatmap (Colour scale by count of applications)
'''

top_10_courses_bar = px.bar(top10_popular_courses_2020, x = 'Applications', y = top10_popular_courses_2020.index, height = 600, color = 'Applications', orientation = 'h',
            color_continuous_scale = ['skyblue','orange'], title = 'Top 10 Most Applications by Course 2020')
top_10_courses_bar.update_layout(barmode='stack', xaxis={'categoryorder':'total ascending'})
top_10_courses_bar.show()

## 5 Contact

While this walkthrough only scratches the surface of the kinds of data processing, analysis and visualisations that can be accomplished we hope this will serve as a jumping off point and template for anyone looking at their own end to end analysis projects or thinking about using Python and/or Jupyter in any aspects of work at UCL. If you have any questions going forward or want to share your own data work then please contact us at the Python Data Working Group:

Sam McIlroy: samuel.mcilroy@ucl.ac.uk<p>
Si Ning Yeoh: s.yeoh@ucl.ac.uk<p>
Oj Akhigbe: oj.akhigbe@ucl.ac.uk